In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets



Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
dataset = load_dataset("imdb")  # automatically downloads train & test
train_data = dataset['train']
test_data = dataset['test']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # start small
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="steps", # Corrected argument name
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    learning_rate=2e-5,
    load_best_model_at_end=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhiijeetdev (abhiijeetdev-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,0.563800,0.359762
200,0.338700,0.371259
300,0.357000,0.318325
400,0.323400,0.298131
500,0.340800,0.281969
600,0.344300,0.267246
700,0.274400,0.281649
800,0.267800,0.298387
900,0.306700,0.401477
1000,0.338900,0.281621


TrainOutput(global_step=3125, training_loss=0.2933514025878906, metrics={'train_runtime': 12698.069, 'train_samples_per_second': 1.969, 'train_steps_per_second': 0.246, 'total_flos': 3311684966400000.0, 'train_loss': 0.2933514025878906, 'epoch': 1.0})

In [ ]:
model.save_pretrained("./my_imdb_model")
tokenizer.save_pretrained("./my_imdb_model")

('./my_imdb_model/tokenizer_config.json',
 './my_imdb_model/special_tokens_map.json',
 './my_imdb_model/vocab.txt',
 './my_imdb_model/added_tokens.json',
 './my_imdb_model/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load your trained model
model_path = "./my_imdb_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(torch.softmax(outputs.logits, dim=1)).item()
    return prediction

# List of tricky sentences
tricky_sentences = [
    "I loved the concept, but hated the execution.",
    "It was okay, not terrible but not great either.",
    "I laughed sometimes, but mostly I was bored.",
    "The movie starts amazing and ends terribly.",
    "The food looked good but tasted weird."
]

print("Testing tricky sentences...\n")
for s in tricky_sentences:
    pred = predict_sentiment(s)
    label = "Positive" if pred == 1 else "Negative"
    print(f"Sentence: {s}")
    print(f"Prediction: {label}")
    print("-"*50)


Testing tricky sentences...

Sentence: I loved the concept, but hated the execution.
Prediction: Negative
--------------------------------------------------
Sentence: It was okay, not terrible but not great either.
Prediction: Negative
--------------------------------------------------
Sentence: I laughed sometimes, but mostly I was bored.
Prediction: Negative
--------------------------------------------------
Sentence: The movie starts amazing and ends terribly.
Prediction: Negative
--------------------------------------------------
Sentence: The food looked good but tasted weird.
Prediction: Negative
--------------------------------------------------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save your model to Drive
model.save_pretrained("/content/drive/MyDrive/my_imdb_model")
tokenizer.save_pretrained("/content/drive/MyDrive/my_imdb_model")


Mounted at /content/drive


('/content/drive/MyDrive/my_imdb_model/tokenizer_config.json',
 '/content/drive/MyDrive/my_imdb_model/special_tokens_map.json',
 '/content/drive/MyDrive/my_imdb_model/vocab.txt',
 '/content/drive/MyDrive/my_imdb_model/added_tokens.json',
 '/content/drive/MyDrive/my_imdb_model/tokenizer.json')

In [ ]:
# Push the committed changes to the remote repository
!git push origin main # Assuming your branch is 'main'. If it's 'master', change 'main' to 'master'.

print("Changes pushed to GitHub.")

fatal: not a git repository (or any of the parent directories): .git
Changes pushed to GitHub.


In [ ]:
# Configure Git to cache credentials in memory for a short time
!git config --global credential.helper store
print("Git credential helper configured. You might be prompted for your GitHub username and password or a Personal Access Token when you push.")

Git credential helper configured. You might be prompted for your GitHub username and password or a Personal Access Token when you push.


In [ ]:
# Push the committed changes to the remote repository
!git push origin main # Assuming your branch is 'main'. If it's 'master', change 'main' to 'master'.

print("Changes pushed to GitHub.")

fatal: not a git repository (or any of the parent directories): .git
Changes pushed to GitHub.


In [ ]:
import os

# Change directory to your cloned repository
%cd ./my_imdb_model/

# Add all changes to the staging area
!git add .

# Commit the changes
# Replace "Add initial project files and trained model" with a more specific message if you like
!git commit -m "Add initial project files and trained model"

print("Changes added and committed.")

[Errno 2] No such file or directory: './my_imdb_model/'
/content
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
Changes added and committed.


In [ ]:
import os
import shutil
import glob

# Define the destination directory (your cloned repository)
repo_dir = './my_imdb_model'

# Find the current notebook file
# Colab notebooks have a specific naming convention or can be found in the current directory
# We'll look for .ipynb files, assuming there's only one main notebook for the project
notebook_files = glob.glob('*.ipynb')
current_notebook = notebook_files[0] if notebook_files else None

if current_notebook:
    # Copy the notebook file to the repository directory
    shutil.copy(current_notebook, repo_dir)
    print(f"Copied notebook: {current_notebook} to {repo_dir}")
else:
    print("Could not find the current notebook file (.ipynb). Please manually copy it to the repository directory.")

# Define the source directory of your saved model
# Based on previous steps, the model was saved in "./results/checkpoint-3125"
model_source_dir = "./results/checkpoint-3125"
model_dest_dir = os.path.join(repo_dir, "my_imdb_model") # Create a subdirectory in the repo for the model

# Check if the model source directory exists
if os.path.exists(model_source_dir):
    # Copy the model directory to the repository directory
    # Use copytree to copy the directory and its contents
    shutil.copytree(model_source_dir, model_dest_dir, dirs_exist_ok=True)
    print(f"Copied model directory: {model_source_dir} to {model_dest_dir}")
else:
    print(f"Model source directory not found at {model_source_dir}. Please verify the path where you saved your model.")

print("Finished copying project files.")

Could not find the current notebook file (.ipynb). Please manually copy it to the repository directory.
Model source directory not found at ./results/checkpoint-3125. Please verify the path where you saved your model.
Finished copying project files.


In [ ]:
# Clone your GitHub repository
!git clone https://github.com/abhiijeetdev/my_imdb_model.git
print("Repository cloned successfully.")

Cloning into 'my_imdb_model'...
fatal: could not read Username for 'https://github.com': No such device or address
Repository cloned successfully.
